In [1]:
from AlgoritmoGenetico import GeneticAlgorithm
import os
import pygame
from main import Game

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Parámetros del algoritmo genético
population_size = 30
mutation_rate = 0.01
num_generations = 300
input_size = 12  # Tamaño del vector de estado
output_size = 4  # Cuatro acciones posibles: izquierda, derecha, disparar, nada

os.environ["SDL_VIDEODRIVER"] = "dummy"  # Configurar el driver de video en "dummy"

In [ ]:
pygame.init()  # Asegúrate de inicializar Pygame
pygame.font.init()


gen_algo = GeneticAlgorithm(population_size, mutation_rate, num_generations, input_size, output_size)

print("Entrenando la red neuronal mediante algoritmo genético...")
trained_model = gen_algo.evolve()  # Este será el mejor modelo entrenado

trained_model.save("resultado_1.keras")

scores = gen_algo.best_scores
rewards =  gen_algo.best_total_scores
epsilons = gen_algo.best_epsilons

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando la red neuronal mediante algoritmo genético...
Inciando generacion 0
Agente 0 Iniciado. Level: 2
Agente 1 Iniciado. Level: 2
Agente 2 Iniciado. Level: 2
Agente 3 Iniciado. Level: 2
Agente 4 Iniciado. Level: 2
Agente 5 Iniciado. Level: 2
Agente 6 Iniciado. Level: 2
Agente 7 Iniciado. Level: 2
Agente 8 Iniciado. Level: 2
Agente 9 Iniciado. Level: 2
Agente 10 Iniciado. Level: 2
Agente 11 Iniciado. Level: 2
Agente 12 Iniciado. Level: 2
Agente 13 Iniciado. Level: 2
Agente 14 Iniciado. Level: 2
Agente 15 Iniciado. Level: 2
Agente 16 Iniciado. Level: 2
Agente 17 Iniciado. Level: 2
Agente 18 Iniciado. Level: 2
Agente 19 Iniciado. Level: 2
Agente 20 Iniciado. Level: 2
Agente 21 Iniciado. Level: 2
Agente 22 Iniciado. Level: 2
Agente 23 Iniciado. Level: 2
Agente 24 Iniciado. Level: 2
Agente 25 Iniciado. Level: 2
Agente 26 Iniciado. Level: 2
Agente 27 Iniciado. Level: 2
Agente 28 Iniciado. Level: 2
Agente 29 Iniciado. Level: 2
-1 Vida. Te quedan 2
Time Finished
Puntaje200, Fitness: -800

In [ ]:
""""
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, num_generations, input_size, output_size, game):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.num_generations = num_generations
        self.input_size = input_size
        self.output_size = output_size
        self.game = game  # La instancia del juego para evaluar cada red
        self.population = self.initialize_population()

    def build_model(self):
        """Construye un modelo de red neuronal."""
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.input_size,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(self.output_size, activation='softmax')
        ])
        return model

    def initialize_population(self):
        """Inicializa una población de redes con pesos aleatorios."""
        population = []
        for _ in range(self.population_size):
            model = self.build_model()
            weights = [layer.get_weights() for layer in model.layers]
            population.append(weights)
        return population

    def evaluate_fitness(self, weights, instance_id, render=True):
        """Evalúa la aptitud de una red neuronal ejecutándola en el juego."""
        from main import Game
        game_instance = Game(800, 600, render=render)  # Cada agente tiene su propia instancia de Game

        print(f'Agente {instance_id} Iniciado')
        
        # Configura el modelo con los pesos dados
        model = self.build_model()
        for layer, weight in zip(model.layers, weights):
            layer.set_weights(weight)

        total_score = 0
        game_instance.reset()
        clock = pygame.time.Clock()
        done = False

        while not done:
            # Lógica del juego y predicción de acción
            game_state = game_instance.get_game_state()
            input_data = self.preprocess_game_state(game_state).reshape(1, -1)
            action = np.argmax(model.predict(input_data))
            
            _, reward, done = game_instance.step(action)
            total_score += reward

            if(reward != 0):
                print(f'Agente {instance_id} Jugo {action} y consiguio una recompensa de {reward}')
            if(action ==3):
                print(f'Agente {instance_id} disparo')
            
            game_instance.run()  # Solo ejecuta run si render está activado

            clock.tick(120)  # Controla la velocidad del bucle

        return total_score

    def preprocess_game_state(self, game_state):
        """Preprocesa el estado del juego para convertirlo en una entrada válida para el modelo."""
        player_pos = np.array(game_state['player_position'])
        alien_pos, alien_distance = game_state['closest_alien']
        laser_pos, laser_distance = game_state['closest_alien_laser']
        obstacle_pos, obstacle_distance = game_state['closest_obstacle']
        
        alien_pos = np.array(alien_pos) if alien_pos else np.zeros(2)
        laser_pos = np.array(laser_pos) if laser_pos else np.zeros(2)
        obstacle_pos = np.array(obstacle_pos) if obstacle_pos else np.zeros(2)
        
        input_data = np.concatenate([player_pos, alien_pos, [alien_distance], laser_pos, [laser_distance], obstacle_pos, [obstacle_distance]])
        return input_data

    def select_best_individuals(self, fitness_scores, num_best=2):
        """Selecciona los mejores individuos basándose en sus puntajes de aptitud."""
        best_indices = np.argsort(fitness_scores)[-num_best:]
        return [self.population[i] for i in best_indices]

    def crossover(self, parent1, parent2):
        """Realiza cruce entre dos individuos para producir un nuevo conjunto de pesos."""
        child = []
        for p1_layer, p2_layer in zip(parent1, parent2):
            new_weights = [(p1 + p2) / 2 for p1, p2 in zip(p1_layer, p2_layer)]
            child.append(new_weights)
        return child

    def mutate(self, weights):
        """Aplica mutación aleatoria a un conjunto de pesos."""
        for layer in weights:
            for i in range(len(layer)):
                if np.random.rand() < self.mutation_rate:
                    layer[i] += np.random.normal()
        return weights

    def create_new_generation(self, best_individuals):
        """Crea una nueva generación usando los mejores individuos y aplicando cruce y mutación."""
        new_population = []
        while len(new_population) < self.population_size:
            parent1, parent2 = random.sample(best_individuals, 2)
            child = self.crossover(parent1, parent2)
            child = self.mutate(child)
            new_population.append(child)
        return new_population

    def evolve(self):
        """Ejecuta el ciclo de evolución a través de varias generaciones."""
        for generation in range(self.num_generations):
            print(f'Inciando generacion {generation}')

            fitness_scores = []

            # Evaluar el primer agente con renderizado activado
            #fitness_score = self.evaluate_fitness(self.population[0], instance_id=0, render=True)
            #fitness_scores.append(fitness_score)

            # Evaluar el resto de los agentes sin renderizado
            with ThreadPoolExecutor(max_workers=self.population_size) as executor:
                args = [(self.population[i], i, False) for i in range(0, self.population_size)]
                other_fitness_scores = list(executor.map(lambda arg: self.evaluate_fitness(*arg), args))
                fitness_scores.extend(other_fitness_scores)

            print(f'Socres de la generacion {generation}: {fitness_scores}')
            # Seleccionar los mejores individuos y generar la nueva población
            best_individuals = self.select_best_individuals(fitness_scores, num_best=5)
            self.population = self.create_new_generation(best_individuals)

            print(f"Generación {generation}: Mejor puntaje = {max(fitness_scores)}")

        # Obtener el mejor modelo entrenado
        best_weights = self.select_best_individuals(fitness_scores, num_best=1)[0]
        final_model = self.build_model()
        for layer, weight in zip(final_model.layers, best_weights):
            layer.set_weights(weight)

        return final_model
""""

In [ ]:
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, num_generations, input_size, output_size):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.num_generations = num_generations
        self.input_size = input_size
        self.output_size = output_size
        self.population = self.initialize_population()
        self.level = 1
        self.win_count = 0

    def build_model(self):
        """Construye un modelo de red neuronal."""
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.input_size,)),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(self.output_size, activation='softmax')
        ])
        return model

    def initialize_population(self):
        """Inicializa una población de redes con pesos aleatorios."""
        population = []
        for _ in range(self.population_size):
            model = self.build_model()
            weights = [layer.get_weights() for layer in model.layers]
            population.append(weights)
        return population

    def evaluate_fitness(self,  weights, generation, instance_id):
        """Evalúa la aptitud de una red neuronal ejecutándola en el juego."""

        print(f'Agente {instance_id} Iniciado. Level: {self.level}')
        
        game = Game(800, 600)

        model = self.build_model()
        for layer, weight in zip(model.layers, weights):
            layer.set_weights(weight)
        
        total_score = 0
        game.reset()  # Reinicia el juego
        clock = pygame.time.Clock()
        done = False

        if self.level == 1:
            shoot = False
            shoot_speed = 0
            time_limit= 60

            if generation > 15:
                time_limit = 120

            elif generation > 10:
                time_limit = 90
        
        elif self.level == 2:
            shoot = True
            shoot_speed = 600
            time_limit = 120

        elif self.level == 3:
            shoot = True
            shoot_speed = 300
            time_limit = 180

        """
        if generation > 0:
            shoot = True
        else:
            shoot = False
        
        if generation > 15:
            shoot_speed = 200
        elif generation > 10:
            shoot_speed = 400
        else:
            shoot_speed= 600


        start_time = time.time()  # Iniciar el temporizador

        if generation > 10:
            time_limit = 180
        elif generation > 5:
            time_limit = 120
        elif generation > 2:
            time_limit = 60
        else:
            time_limit = 30
        """

        start_time = time.time()  # Iniciar el temporizador

        pygame.font.init()
        font = pygame.font.Font(None, 36)  # Fuente predeterminada, tamaño 36

        ALIENLASER = pygame.USEREVENT + 1
        pygame.time.set_timer(ALIENLASER, shoot_speed)

        while not done:
            elapsed_time = int(time.time() - start_time)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

                if event.type == ALIENLASER and shoot == True:
                    game.alien_shoot()

            game_state = game.get_game_state()
            input_data = self.preprocess_game_state(game_state).reshape(1, -1)

            epsilon = 0.1  # Probabilidad de explorar aleatoriamente

            # Obtener las probabilidades de las acciones
            action_probs = model.predict(input_data, verbose=0)
            print(action_probs)

            # Decidir si explorar o explotar
            if np.random.rand() < epsilon:
                # Exploración: elegir una acción aleatoria
                action = np.random.choice(len(action_probs))
            else:
                # Explotación: elegir la acción con la mayor probabilidad
                action = np.argmax(action_probs) 

            _, reward, done = game.step(action, start_time)
            total_score += reward

            game.screen.fill((30, 30, 30))
            game.run()

            timer_text = font.render(f"Tiempo: {elapsed_time}s", True, (255, 255, 255))
            text_rect = timer_text.get_rect(center=(game.screen.get_width() // 2, 20))
            game.screen.blit(timer_text, text_rect)

            pygame.display.flip()
            clock.tick(120)

            if elapsed_time > time_limit:
                print("Time Finished")
                done = True
                total_score -= 1000
            
            if not game.aliens.sprites():
                print("Ganaste")
                self.win_count += 1
                if self.win_count >= 5 and self.level != 3:
                    self.level += 1
                    self.win_count = 0
                done = True

        print(total_score)
        return total_score

    def preprocess_game_state(self, game_state):
        """Preprocesa el estado del juego para convertirlo en una entrada válida para el modelo."""
        player_pos = np.array(game_state['player_position'])
        alien_pos, alien_distance = game_state['closest_alien']
        laser_pos, laser_distance = game_state['closest_alien_laser']
        obstacle_pos, obstacle_distance = game_state['closest_obstacle']
        
        alien_pos = np.array(alien_pos) if alien_pos else np.zeros(2)
        laser_pos = np.array(laser_pos) if laser_pos else np.zeros(2)
        obstacle_pos = np.array(obstacle_pos) if obstacle_pos else np.zeros(2)
        
        input_data = np.concatenate([player_pos, alien_pos, [alien_distance], laser_pos, [laser_distance], obstacle_pos, [obstacle_distance]])
        return input_data

    def select_best_individuals(self, fitness_scores, num_best=2):
        """Selecciona los mejores individuos basándose en sus puntajes de aptitud."""
        best_indices = np.argsort(fitness_scores)[-num_best:]
        return [self.population[i] for i in best_indices]

    def crossover(self, parent1, parent2):
        """Realiza cruce entre dos individuos para producir un nuevo conjunto de pesos."""
        child = []
        for p1_layer, p2_layer in zip(parent1, parent2):
            new_weights = [(p1 + p2) / 2 for p1, p2 in zip(p1_layer, p2_layer)]
            child.append(new_weights)
        return child

    def mutate(self, weights):
        """Aplica mutación aleatoria a un conjunto de pesos."""
        for layer in weights:
            for i in range(len(layer)):
                if np.random.rand() < self.mutation_rate:
                    layer[i] += np.random.normal()
        return weights

    def create_new_generation(self, best_individuals):
        """Crea una nueva generación usando los mejores individuos y aplicando cruce y mutación."""
        new_population = []
        while len(new_population) < self.population_size:
            parent1, parent2 = random.sample(best_individuals, 2)
            child = self.crossover(parent1, parent2)
            child = self.mutate(child)
            new_population.append(child)
        return new_population

    def evolve(self):
        """Ejecuta el ciclo de evolución a través de varias generaciones."""
        for generation in range(self.num_generations):
            print(f'Inciando generacion {generation}')

            fitness_scores = []
    
                # Crea un ThreadPoolExecutor con tantos hilos como tamaño de la población
            with ThreadPoolExecutor(max_workers=self.population_size) as executor:
                # Define una lista de tareas para cada modelo en la población, asociada a su instancia de juego
                futures = [
                    executor.submit(self.evaluate_fitness, weight, generation, i)
                    for i, weight in enumerate(self.population)
                ]
                
                # Recolecta los resultados a medida que se completan las tareas
                for future in as_completed(futures):
                    fitness_scores.append(future.result())

            print(f'Socres de la generacion {generation}: {fitness_scores}')
            # Seleccionar los mejores individuos y generar la nueva población
            best_individuals = self.select_best_individuals(fitness_scores, num_best=5)
            self.population = self.create_new_generation(best_individuals)

            print(f"Generación {generation}: Mejor puntaje = {max(fitness_scores)}")

        # Obtener el mejor modelo entrenado
        best_weights = self.select_best_individuals(fitness_scores, num_best=1)[0]
        final_model = self.build_model()
        for layer, weight in zip(final_model.layers, best_weights):
            layer.set_weights(weight)

        return final_model